In [1]:
from dataLoader import PointCloudLoader
import torch
from net import RadarNet
from trainer import train

In [2]:
# 总类别数
num_classes = 14
# 学习率
lr = 1e-6
# 训练轮数
epoch = 100
# 每个点云数据从中取num_len个区域
num_len = 6
# 每个区域的大小
area_size = (128,128)
# 最高高程长度num_z个
num_z = 300
# 平面和高程分辨
xy_resolution = 0.5
z_resolution = 0.5
# 编码长度
embed_size = 32
# 隐变量长度
num_hiddens = 32
# 循环神经网络层数
num_layers = 2
# 训练阶段每一次输入的序列数量(分块计算会减少显存使用，但是会减速，仅在训练阶段有意义)
num_seq = 16*16*4*4
# num_seq = None

train_dir = "./powerline_chunk/train"
test_dir = "./powerline_chunk/test"
device = "cuda"

In [3]:
# 训练集要打乱，每一块生成num_len个随机区域，一共是总文件数*num_len个
train_dataloader = PointCloudLoader(
    root_path=train_dir,
    num_classes=num_classes,
    num_len=num_len,
    area_size=area_size,
    num_z=num_z,
    xy_resolution=xy_resolution,
    z_resolution=z_resolution,
    random=True,
)
# 测试集不打乱，按顺序来一块一块分类
test_dataloader = PointCloudLoader(
    root_path=test_dir,
    num_classes=num_classes,
    num_len=num_len,
    area_size=area_size,
    num_z=num_z,
    xy_resolution=xy_resolution,
    z_resolution=z_resolution,
)

test_loader = torch.utils.data.DataLoader(test_dataloader, batch_size=1)
train_loader = torch.utils.data.DataLoader(train_dataloader, batch_size=1)

net = RadarNet(
    num_classes=num_classes,
    elevation_resolution=num_z,
    embed_size=embed_size,
    num_hiddens=num_hiddens,
    num_layers=num_layers,
    num_seq=num_seq,
    features=[64, 128, 256, 512, 1024],
    # features=[128, 256, 512, 1024, 1024],
    dropout=0,
)
weight = torch.load("best_model_train.pth")
net.load_state_dict(weight,strict=False)

optimizer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=1e-5)

In [4]:
from utils import pred_file,save_result
pred, header = pred_file(
    net,
    "powerline/train/GT2_01.las",
    area_size,
    num_z,
    xy_resolution,
    z_resolution,
    resolution=100,
    device=device,
    only_pred=True,
)
save_result(pred, "pred.las", header)

In [4]:
from utils import eval_net_point
eval_net_point(net,test_loader,device)

-----------------Eval Point--------------


Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Valid acc:     | 0.397 | 0.817 | 0.976 | 0.726 | 0.457 | 0.000 | 0.951 | 0.869 | 0.311 | 0.914 | 0.354 | 0.000 | 0.762 | 0.680 | 0.587
Valid recall:  | 0.158 | 0.903 | 0.978 | 0.980 | 0.919 | 0.000 | 0.961 | 0.929 | 0.743 | 0.954 | 0.769 | 0.000 | 0.836 | 0.921 | 0.718
Valid iou:     | 0.120 | 0.755 | 0.956 | 0.715 | 0.435 | 0.000 | 0.921 | 0.816 | 0.277 | 0.877 | 0.315 | 0.000 | 0.666 | 0.644 | 0.535


(0.5867208516854054, 0.7178706126514915, 0.5354981674710885)

In [5]:
label_weights = torch.tensor(
    [
        1.0000,
        1.9758,
        1.2000,
        1.0000,
        14.4916,
        15.8751,
        10.2944,
        3.8078,
        5.4598,
        13.3421,
        7.7485,
        19.1769,
        130.2272,
        11.7641,
        10.9375,
    ]
)
# label_weights = train_dataloader.get_weight()

In [6]:
train(
    net=net,
    optimizer=optimizer,
    test_loader=test_loader,
    train_loader=train_loader,
    epoch=epoch,
    num_len=len(train_dataloader.files) * num_len,
    device=device,
    classes_weights=label_weights,
    area_size=area_size,
)

Running on: cuda
Net Statistics:
[INFO] Register count_gru() for <class 'torch.nn.modules.rnn.GRU'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
    FLOPs: 83.456294912 G
    Params: 40.193807 M
----------------------------------
FPS Calculating on cuda
    FPS: 8.036943500205028
    InputSize: torch.Size([1, 100, 128, 128])
----------------------------------
-----------------Epoch:1-----------------


epoch:1, loss:0.80026, mIOU:0.72571
Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Train acc:     | 0.545 | 0.852 | 0.978 | 0.777 | 0.824 | 0.898 | 0.976 | 0.850 | 0.701 | 0.972 | 0.686 | 0.495 | 0.725 | 0.696 | 0.784
Train recall:  | 0.489 | 0.860 | 0.974 | 0.931 | 0.941 | 0.947 | 0.981 | 0.912 | 0.914 | 0.979 | 0.930 | 0.959 | 0.806 | 0.880 | 0.893
Train iou:     | 0.343 | 0.751 | 0.953 | 0.730 | 0.785 | 0.851 | 0.960 | 0.787 | 0.666 | 0.954 | 0.657 | 0.474 | 0.616 | 0.632 | 0.726
-----------------Eval Point--------------


Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Valid acc:     | 0.396 | 0.815 | 0.977 | 0.730 | 0.455 | 0.000 | 0.948 | 0.871 | 0.281 | 0.901 | 0.340 | 0.000 | 0.740 | 0.650 | 0.579
Valid recall:  | 0.160 | 0.904 | 0.976 | 0.979 | 0.920 | 0.000 | 0.965 | 0.936 | 0.744 | 0.958 | 0.799 | 0.000 | 0.845 | 0.914 | 0.722
Valid iou:     | 0.121 | 0.754 | 0.955 | 0.719 | 0.435 | 0.000 | 0.922 | 0.823 | 0.254 | 0.868 | 0.311 | 0.000 | 0.660 | 0.614 | 0.531
-----------------Epoch:2-----------------


epoch:2, loss:0.79589, mIOU:0.72025
Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Train acc:     | 0.546 | 0.851 | 0.978 | 0.774 | 0.821 | 0.919 | 0.979 | 0.846 | 0.699 | 0.974 | 0.629 | 0.488 | 0.721 | 0.702 | 0.781
Train recall:  | 0.491 | 0.860 | 0.976 | 0.887 | 0.947 | 0.936 | 0.983 | 0.902 | 0.908 | 0.982 | 0.861 | 0.938 | 0.814 | 0.902 | 0.885
Train iou:     | 0.344 | 0.751 | 0.956 | 0.699 | 0.788 | 0.867 | 0.964 | 0.778 | 0.661 | 0.958 | 0.580 | 0.468 | 0.618 | 0.652 | 0.720
-----------------Epoch:3-----------------


epoch:3, loss:0.81394, mIOU:0.73209
Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Train acc:     | 0.550 | 0.851 | 0.978 | 0.807 | 0.813 | 0.911 | 0.977 | 0.839 | 0.669 | 0.973 | 0.665 | 0.503 | 0.754 | 0.706 | 0.785
Train recall:  | 0.496 | 0.856 | 0.976 | 0.952 | 0.938 | 0.952 | 0.983 | 0.898 | 0.901 | 0.979 | 0.942 | 0.979 | 0.833 | 0.913 | 0.900
Train iou:     | 0.349 | 0.747 | 0.955 | 0.780 | 0.774 | 0.871 | 0.962 | 0.771 | 0.630 | 0.956 | 0.639 | 0.494 | 0.658 | 0.662 | 0.732
-----------------Epoch:4-----------------


...

epoch:23, loss:0.78889, mIOU:0.73073
Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Train acc:     | 0.550 | 0.852 | 0.978 | 0.778 | 0.821 | 0.926 | 0.976 | 0.849 | 0.666 | 0.974 | 0.669 | 0.498 | 0.729 | 0.709 | 0.784
Train recall:  | 0.499 | 0.856 | 0.974 | 0.953 | 0.945 | 0.950 | 0.984 | 0.903 | 0.909 | 0.982 | 0.934 | 0.991 | 0.829 | 0.904 | 0.901
Train iou:     | 0.350 | 0.748 | 0.954 | 0.747 | 0.785 | 0.884 | 0.962 | 0.778 | 0.639 | 0.958 | 0.639 | 0.496 | 0.633 | 0.658 | 0.731
-----------------Epoch:24-----------------


epoch:24, loss:0.80220, mIOU:0.73559
Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Train acc:     | 0.548 | 0.852 | 0.978 | 0.821 | 0.815 | 0.936 | 0.981 | 0.837 | 0.699 | 0.975 | 0.698 | 0.480 | 0.737 | 0.701 | 0.790
Train recall:  | 0.493 | 0.858 | 0.976 | 0.927 | 0.944 | 0.979 | 0.985 | 0.885 | 0.914 | 0.984 | 0.943 | 0.977 | 0.822 | 0.892 | 0.899
Train iou:     | 0.346 | 0.749 | 0.955 | 0.770 | 0.779 | 0.917 | 0.967 | 0.764 | 0.664 | 0.961 | 0.676 | 0.473 | 0.636 | 0.640 | 0.736
-----------------Epoch:25-----------------


epoch:25, loss:0.79556, mIOU:0.74111
Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Train acc:     | 0.553 | 0.848 | 0.977 | 0.841 | 0.817 | 0.910 | 0.977 | 0.837 | 0.722 | 0.975 | 0.724 | 0.552 | 0.722 | 0.690 | 0.796
Train recall:  | 0.508 | 0.853 | 0.973 | 0.946 | 0.950 | 0.950 | 0.982 | 0.892 | 0.924 | 0.978 | 0.955 | 0.926 | 0.825 | 0.898 | 0.897
Train iou:     | 0.356 | 0.743 | 0.951 | 0.796 | 0.785 | 0.874 | 0.961 | 0.764 | 0.691 | 0.956 | 0.702 | 0.526 | 0.628 | 0.640 | 0.741
-----------------Eval Point--------------


Class:         |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 |    13 |    14 |   AVG
Valid acc:     | 0.398 | 0.816 | 0.977 | 0.710 | 0.467 | 0.000 | 0.952 | 0.876 | 0.283 | 0.915 | 0.359 | 0.000 | 0.764 | 0.665 | 0.584
Valid recall:  | 0.162 | 0.902 | 0.977 | 0.983 | 0.921 | 0.000 | 0.968 | 0.931 | 0.761 | 0.961 | 0.822 | 0.000 | 0.844 | 0.922 | 0.725
Valid iou:     | 0.122 | 0.754 | 0.956 | 0.702 | 0.447 | 0.000 | 0.927 | 0.824 | 0.260 | 0.883 | 0.330 | 0.000 | 0.672 | 0.630 | 0.536
-----------------Epoch:26-----------------


KeyboardInterrupt: 